In [125]:
from pymongo import MongoClient
import pandas as pd


client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### Startup?

Determining whether a company is a startup typically involves considering various factors, and it may not always be straightforward from the data you've provided. However, you can make an educated assessment by looking at certain indicators. Here are some common criteria to consider:

**1. Founding Year:** Check the "founded_year" field in the document. If the company was founded relatively recently, it's more likely to be a startup. However, there is no strict definition of when a company ceases to be a startup, and the threshold can vary depending on context. Some consider companies as startups for the first few years, while others may extend this period.

**2 .Funding Rounds:** Look for information on funding rounds in the document. Startups often go through multiple rounds of funding to grow their business. If you see early-stage funding rounds (e.g., Series A, Series B), it's a sign that the company is still in a growth phase. However, some mature companies may also receive funding.

**3. Number of Employees:** Consider the "number_of_employees" field. Startups typically start with a small team and gradually expand as they grow. A low number of employees can be an indicator of a startup, but this can vary depending on the industry and business model.

**4. Acquisition Status:** Check for information on acquisitions. If the company has been acquired recently by a larger corporation, it might no longer be considered a startup. However, some startups continue to operate independently after being acquired.

In [181]:
def search_startups(country_code, min_founded_year, min_rised_amount, min_founding_rounds=1, max_employees=100, been_acquired=True):

    pipeline = [
        {
            "$match": {
                "founded_year": {"$gt": min_founded_year},
                "offices.country_code": country_code,
                "acquisition": {"$exists": been_acquired},
                "number_of_employees": {"$lte": max_employees}
            }
        },
        {
            "$unwind": "$funding_rounds"
        },
        {
            "$group": {
                "_id": "$_id",
                "name": {"$first": "$name"},
                "founded_year": {"$first": "$founded_year"},
                "funding_rounds": {"$push": "$funding_rounds"},
                "total_funding_amount": {"$sum": "$funding_rounds.raised_amount"},
                "unique_currency_codes": {"$addToSet": "$funding_rounds.raised_currency_code"},
                "total_employees": {"$first": "$number_of_employees"},
                "offices": {"$first": "$offices"}
            }
        },
        {
            "$match": {
                "total_funding_amount": {"$gte": min_rised_amount},
                "$expr": {
                    "$gte": [
                        {"$size": "$funding_rounds"},
                        min_founding_rounds
                    ]
                }
            }
        },
        {
            "$project": {
                "_id": 0,
                "Company": "$name",
                "Founded year": "$founded_year",
                "Total Funding Amount": "$total_funding_amount",
                "Funding Rounds": {"$size": "$funding_rounds"},
                "Funding currency code": "$unique_currency_codes",
                "Total Employees": "$total_employees",
                "First Office": {"$arrayElemAt": ["$offices", 0]},
                "Office City": {"$arrayElemAt": ["$offices.city", 0]},
                "First Office Location": {
                    "Latitude": {"$arrayElemAt": ["$offices.latitude", 0]},
                    "Longitude": {"$arrayElemAt": ["$offices.longitude", 0]}
                }
            }
        }
    ]

    result = list(c.aggregate(pipeline))
    df = pd.DataFrame(result)
    return result, df

In [190]:
# Last company founded is from 2013 so assuming no more data was collected since that year well set a company in its ealry stages at year
# result is json
# by the info in the df, San Francisco have more options
result , df =  search_startups("USA", 2007, 1000000, 1, 100)

df

,Company,Founded year,Total Funding Amount,Funding Rounds,Funding currency code,Total Employees,First Office,Office City,First Office Location
0,AdRocket,2008,1000000,1,[USD],5,"{'description': 'AdRocket Headquarters', 'addr...",San Francisco,"{'Latitude': 37.775196, 'Longitude': -122.419204}"
1,Open Places,2008,1100000,4,[USD],3,"{'description': 'San Francisco Office', 'addre...",San Francisco,"{'Latitude': 37.758352, 'Longitude': -122.398379}"
2,Home-Account,2008,1000000,2,[USD],10,"{'description': '', 'address1': '188 King Stre...",San Francisco,"{'Latitude': 37.778551, 'Longitude': -122.391686}"
3,EQAL,2008,6875000,2,[USD],75,"{'description': '', 'address1': '5250 Lankersh...",North Hollywood,"{'Latitude': None, 'Longitude': None}"
4,Mashape,2010,1601000,2,[USD],15,"{'description': 'Mashape Lair', 'address1': '4...",San Francisco,"{'Latitude': None, 'Longitude': None}"
...,...,...,...,...,...,...,...,...,...
198,Foodista,2008,1300000,2,[USD],3,"{'description': '', 'address1': '24 Roy Street...",Seattle,"{'Latitude': 47.6254784, 'Longitude': -122.356..."
199,oneforty,2009,2345000,4,[USD],5,"{'description': 'Main office', 'address1': '61...",Cambridge,"{'Latitude': 42.364014, 'Longitude': -71.101375}"
200,Stack Exchange,2008,18000000,2,[USD],100,"{'description': 'New York Office', 'address1':...",New York,"{'Latitude': None, 'Longitude': None}"
201,Diffbot,2010,2000000,1,[USD],6,"{'description': '', 'address1': '395 Page Mill...",Palo Alto,"{'Latitude': None, 'Longitude': None}"


In [173]:
result

[{'Company': 'Emotive Communications',
  'Total Funding Amount': 13950000,
  'Funding Rounds': 2,
  'Funding currency code': ['USD'],
  'Total Employees': 25,
  'First Office': {'description': 'HQ',
   'address1': '345 California Street',
   'address2': '10th Floor',
   'zip_code': '94104',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': None,
   'longitude': None},
  'Office City': 'San Francisco',
  'First Office Location': {'Latitude': None, 'Longitude': None}},
 {'Company': 'Social Gaming Network',
  'Total Funding Amount': 17100000,
  'Funding Rounds': 4,
  'Funding currency code': ['USD', None],
  'Total Employees': 100,
  'First Office': {'description': 'HQ Los Angeles',
   'address1': '9570 W. Pico Blvd 3rd Floor',
   'address2': '',
   'zip_code': '90035',
   'city': 'Los Angeles',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.446823,
   'longitude': -122.161523},
  'Office City': 'Los Angeles',
  'First Office L